# FFT Convolution Demo

In [1]:
import torch
import torch.nn.functional as f

from fft_conv import fft_conv1d, fft_conv2d, fft_conv3d, FFTConv1d

### Example Usage -- 1D Convolutions

In [2]:
# Create dummy data.  
#     Data shape: (batch, channels, length)
#     Kernel shape: (out_channels, in_channels, kernel_size)
# For ordinary 1D convolution, simply set batch=1.
signal = torch.randn(3, 3, 1024 * 1024)
kernel = torch.randn(2, 3, 128)
bias = torch.randn(2)

# Functional execution.  (Easiest for generic use cases.)
out = fft_conv1d(signal, kernel, bias=bias)

# Object-oriented execution.  (Requires some extra work, since the 
# defined classes were designed for use in neural networks.)
fft_conv = FFTConv1d(3, 2, 128, bias=True)
fft_conv.weight = torch.nn.Parameter(kernel)
fft_conv.bias = torch.nn.Parameter(bias)
out = fft_conv(signal)

### 1D Test

In [3]:
# Create dummy data
kernel_size = 255
padding = kernel_size // 2
kernel = torch.randn(2, 3, kernel_size)
signal = torch.randn(3, 3, 1024 * 1024)
bias = torch.randn(2)

# Perform both direct and FFT convolutions
print('--- Direct Convolution ---')
%time y0 = f.conv1d(signal, kernel, bias=bias, padding=padding)
print('--- FFT Convolution ---')
%time y1 = fft_conv1d(signal, kernel, bias=bias, padding=padding)

# Print input/output tensor shapes
print(f'\nInput shape: {signal.shape}')
print(f'Output shape: {y0.shape}')

# Compute relative error
abs_error = torch.abs(y0 - y1)
print(f'\nAbs Error Mean: {abs_error.mean():.3E}')
print(f'Abs Error Std Dev: {abs_error.std():.3E}')

--- Direct Convolution ---
CPU times: user 2.79 s, sys: 19.5 ms, total: 2.81 s
Wall time: 482 ms
--- FFT Convolution ---
CPU times: user 1.82 s, sys: 145 ms, total: 1.97 s
Wall time: 383 ms

Input shape: torch.Size([3, 3, 1048576])
Output shape: torch.Size([3, 2, 1048576])

Abs Error Mean: 9.452E-06
Abs Error Std Dev: 7.259E-06


### 2D Test

In [4]:
# Create dummy data
kernel_size = 11
padding = kernel_size // 2
kernel = torch.randn(2, 3, kernel_size, kernel_size)
signal = torch.randn(3, 3, 1024, 1024)
bias = torch.randn(2)

# Perform both direct and FFT convolutions
print('--- Direct Convolution ---')
%time y0 = f.conv2d(signal, kernel, bias=bias, padding=padding)
print('--- FFT Convolution ---')
%time y1 = fft_conv2d(signal, kernel, bias=bias, padding=padding)

# Print input/output tensor shapes
print(f'\nInput shape: {signal.shape}')
print(f'Output shape: {y0.shape}')

# Compute relative error
abs_error = torch.abs(y0 - y1)
print(f'\nAbs Error Mean: {abs_error.mean():.3E}')
print(f'Abs Error Std Dev: {abs_error.std():.3E}')

--- Direct Convolution ---
CPU times: user 1.45 s, sys: 8.59 ms, total: 1.46 s
Wall time: 245 ms
--- FFT Convolution ---
CPU times: user 637 ms, sys: 43.8 ms, total: 681 ms
Wall time: 116 ms

Input shape: torch.Size([3, 3, 1024, 1024])
Output shape: torch.Size([3, 2, 1024, 1024])

Abs Error Mean: 4.636E-06
Abs Error Std Dev: 3.652E-06


### 3D Test

In [6]:
# Create dummy data
kernel_size = 3
padding = kernel_size // 2
kernel = torch.randn(2, 3, kernel_size, kernel_size, kernel_size)
signal = torch.randn(3, 3, 100, 300, 300)
bias = torch.randn(2)

# Perform both direct and FFT convolutions
print('--- Direct Convolution ---')
%time y0 = f.conv3d(signal, kernel, bias=bias, padding=padding)
print('--- FFT Convolution ---')
%time y1 = fft_conv3d(signal, kernel, bias=bias, padding=padding)

# Print input/output tensor shapes
print(f'\nInput shape: {signal.shape}')
print(f'Output shape: {y0.shape}')

abs_error = torch.abs(y0 - y1)
print(f'\nAbs Error Mean: {abs_error.mean():.3E}')
print(f'Abs Error Std Dev: {abs_error.std():.3E}')

--- Direct Convolution ---
CPU times: user 9.44 s, sys: 2.5 s, total: 11.9 s
Wall time: 2.2 s
--- FFT Convolution ---
CPU times: user 4.56 s, sys: 771 ms, total: 5.34 s
Wall time: 903 ms

Input shape: torch.Size([3, 3, 100, 300, 300])
Output shape: torch.Size([3, 2, 100, 300, 300])

Abs Error Mean: 3.715E-06
Abs Error Std Dev: 2.865E-06
